#LangChain + SQL

##Configurazione

In [ ]:
!pip install langchain
!pip install anthropic
!pip install langchain-google-vertexai
!pip install -qU langchain langchain-openai langchain-community langchain-experimental pandas

In [ ]:
import pandas as pd
import os
import logging
import json
import sqlite3
from sqlalchemy import create_engine

from google.auth import credentials
from google.oauth2 import service_account
import google.cloud.aiplatform as aiplatform

from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain_google_vertexai import VertexAI
from langchain_community.agent_toolkits import create_sql_agent
from langchain_experimental.sql import SQLDatabaseChain
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType
from langchain_community.utilities import SQLDatabase
from langchain.prompts.prompt import PromptTemplate
from langchain import LLMChain

import vertexai
from anthropic import AnthropicVertex

In [ ]:
logging.basicConfig(level=logging.ERROR)

In [ ]:
# Lettura JSON e aggiornamento parametri
with open(
    "service_account.json"
) as f:
    credentials = json.load(f)

my_credentials = service_account.Credentials.from_service_account_info(
    credentials
)

# Inizializzazione AI Platform
aiplatform.init(
    credentials=my_credentials,
)

with open("service_account.json", encoding="utf-8") as f:
    project_json = json.load(f)
    project_id = project_json["project_id"]

# Inizializzazione Vertex AI
vertexai.init(project=project_id, location="europe-west1")

In [ ]:
llm = VertexAI(model_name="text-bison@002", max_output_tokens=50)

Convertiamo il csv in una tabella persistente e apriamo la connessione.

## Interroghiamo dataNFC

In [ ]:
# Lettura file CSV
df = pd.read_csv('dataNFC.csv', sep=';')

# Connessione al database SQLite
conn = sqlite3.connect('datanfc.db')

# Salva il DataFrame nel database come tabella
df.to_sql(name="datanfc", con=conn, if_exists='replace', index=False)

# Creazione DB
db = SQLDatabase.from_uri(database_uri="sqlite:///datanfc.db",
                          include_tables=['datanfc'],
                          sample_rows_in_table_info=2)


print(db.dialect)
print(db.get_usable_table_names())
print(db.get_table_info())

sqlite
['datanfc']

CREATE TABLE datanfc (
	status TEXT, 
	systemtime INTEGER, 
	datetime TEXT, 
	bool1 INTEGER
)

/*
2 rows from datanfc table:
status	systemtime	datetime	bool1
Checked In	1689923154	21/07/2023 09:02	1
Checked out	1689933303	21/07/2023 11:55	0
*/


In [ ]:
# Configurazione toolkit
toolkit = SQLDatabaseToolkit(llm=llm, db=db)

# Creazione SQL agent
agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [ ]:
agent.invoke({"input": "How many checked out in august 2023?"})



> Entering new SQL Agent Executor chain...
 The checkout table has the checkout date. I can query the checkout table and filter the checkout date to get the count of checked out books in August 2023.
Action: sql_db_query
Action Input: SELECT     CAST(count(*) as BIGNUMERIC)   FROM     CHECKOUT   WHERE CHECKOUTDATE BETWEEN '08-01-2023' AND '08-31-2023'Error: (sqlite3.OperationalError) no such table: CHECKOUT
[SQL: SELECT     CAST(count(*) as BIGNUMERIC)   FROM     CHECKOUT   WHERE CHECKOUTDATE BETWEEN '08-01-2023' AND '08-31-2023']
(Background on this error at: https://sqlalche.me/e/20/e3q8) There is no CHECKOUT table. I need to find the correct table name.
Action: sql_db_list_tables
Action Input: datanfc The table name is datanfc. I can now query the datanfc table to get the count of checked out books in August 2023.
Action: sql_db_query
Action Input: SELECT     CAST(count(*) as BIGNUMERIC)   FROM     datanfc   WHERE CHECKOUTDATE BETWEEN '08-01-2023' AND '08-31-2023'Error: (sqlite3.O

{'input': 'How many checked out in august 2023?', 'output': '0'}

Per come è strutturato il dataset, l'LLM fa fatica a ricostruire il significato semantico della richiesta rispetto alla struttura dei dati, perchè:
- La colonna ***status*** non riflette appieno la descrizione Checked in/out, a meno che non venga richiesto esplicitamente in questo modo;
- La colonna ***datetime*** contiene informazioni di tipo data e ora non formattate secondo un metodo standard, complicando la comprensione della data, dell'orario e la separazione tra i due.
Questi due motivi portano ad una risposta erronea rispetto alla domanda fornita.

Con questo approccio, è quindi necessario che le colonne riflettano semanticamente i possibili valori che possono assumere, o in alternativa avere una descrizione di ciascuna colonna, e che i dati di tipo ora, data, datetime e timestamp seguano una formattazione standard in modo da facilitare la ricerca delle informazioni.

## Interroghiamo titanic

In [ ]:
# Lettura file CSV
df = pd.read_csv('titanic.csv', sep=',')

# Connessione al database SQLite
conn = sqlite3.connect('titanic.db')

# Salva il DataFrame nel database come tabella
df.to_sql(name="titanic", con=conn, if_exists='replace', index=False)

# Creazione DB
db = SQLDatabase.from_uri(database_uri="sqlite:///titanic.db",
                          include_tables=['titanic'],
                          sample_rows_in_table_info=2)


print(db.dialect)
print(db.get_usable_table_names())
print(db.get_table_info())

sqlite
['titanic']

CREATE TABLE titanic (
	"PassengerId" INTEGER, 
	"Survived" INTEGER, 
	"Pclass" INTEGER, 
	"Name" TEXT, 
	"Sex" TEXT, 
	"Age" REAL, 
	"SibSp" INTEGER, 
	"Parch" INTEGER, 
	"Ticket" TEXT, 
	"Fare" REAL, 
	"Cabin" TEXT, 
	"Embarked" TEXT
)

/*
2 rows from titanic table:
PassengerId	Survived	Pclass	Name	Sex	Age	SibSp	Parch	Ticket	Fare	Cabin	Embarked
1	0	3	Braund, Mr. Owen Harris	male	22.0	1	0	A/5 21171	7.25	None	S
2	1	1	Cumings, Mrs. John Bradley (Florence Briggs Thayer)	female	38.0	1	0	PC 17599	71.2833	C85	C
*/


In [ ]:
# Configurazione toolkit
toolkit = SQLDatabaseToolkit(llm=llm, db=db)

# Configurazione SQL agent
agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [ ]:
agent.invoke({"input": "How many adult male passengers survived?"})



> Entering new SQL Agent Executor chain...
 I need to count the number of adult male passengers who survived.
Action: sql_db_query
Action Input: SELECT COUNT(*) FROM passenger WHERE sex = 'male' AND age >= 18 AND survived = 1;Error: (sqlite3.OperationalError) no such table: passenger
[SQL: SELECT COUNT(*) FROM passenger WHERE sex = 'male' AND age >= 18 AND survived = 1;]
(Background on this error at: https://sqlalche.me/e/20/e3q8) The passenger table does not exist.
Action: sql_db_list_tables
Action Input: titanic The only table in the database is titanic.
Action: sql_db_query
Action Input: SELECT COUNT(*) FROM titanic WHERE sex = 'male' AND age >= 18 AND survived = 1;[(70,)] 70 adult male passengers survived.
Final Answer: 70

> Finished chain.


{'input': 'How many adult male passengers survived?', 'output': '70'}

Come è possibile vedere dal risultato della query, usando un dataset ben strutturato è possibile  ottenere facilmente informazioni dalla base di dati.

## Memoria

Creiamo un esempio di chain con memoria in grado di recuperare le informazioni richieste dall'utente dalla base dati.

In [ ]:
TEMPLATE = """Given the following conversation respond just to the question you were asked using the chat history and the query result as resources.
Don't give informations not required.

Current conversation:
{history}{resources}
Human: {input}
AI Assistant:"""

Definiamo una classe per creare la chain SQL con memoria

In [ ]:
class MySQLAgentWithMemory:
  def __init__(self, llm, toolkit, verbose):
    self.llm = llm
    self.toolkit = toolkit
    self.agent = create_sql_agent(
        llm=llm,
        toolkit=toolkit,
        agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=verbose
    )
    self.history = ConversationChain(
        llm=llm,
        verbose=verbose,
        memory=ConversationBufferMemory()
    )

  def invoke(self, question):

    # Storico conversazione per ricostruire la domanda
    resources = self.agent.invoke("[Previous infromations:" + self.history.memory.buffer[-150:] + "] current question:" + question)

    # Crea il prompt personalizzato per le risorse recuperate e assegnalo alla conversazione
    prompt = PromptTemplate.from_template(TEMPLATE).partial(resources=resources)
    self.history.prompt = prompt

    # Predici la risposta
    answer = self.history.predict(input=question)

    return answer

In [ ]:
agent = MySQLAgentWithMemory(llm, toolkit, verbose=False)

In [ ]:
question = "What was the name of the oldest passenger?"
response = agent.invoke(question)

print(response)

 Barkworth, Mr. Algernon Henry Wilson


In [ ]:
question = "How old was he?"
response = agent.invoke(question)

print(response)

 80 years old.


In [ ]:
question = "Which was his cabin?"
response = agent.invoke(question)

print(response)

 A23


La chain è in grado di rispondere basandosi sul DB e sullo storico della conversazione.